# Classify text with BERT

### Learning Objectives
1. Learn how to load a pre-trained BERT model from KerasHub
2. Learn how to build your own model by combining with a classifier
3. Learn how to train an LLM (BERT) model by fine-tuning
4. Learn how to save your trained model and use it
5. Learn how to evaluate a text classification model

This lab will show you how to fine-tune BERT to perform sentiment analysis on a dataset of plain-text IMDB movie reviews.
In addition to training a model, you will learn how to preprocess text into an appropriate format.

### Before you start

Please ensure you have a GPU (1 x NVIDIA Tesla T4 should be enough) attached to your Notebook instance to ensure that the training doesn't take too long. 

## About BERT

[BERT](https://arxiv.org/abs/1810.04805) and other Transformer encoder architectures have been wildly successful on a variety of tasks in NLP (natural language processing). They compute vector-space representations of natural language that are suitable for use in deep learning models. The BERT family of models uses the Transformer encoder architecture to process each token of input text in the full context of all tokens before and after, hence the name: Bidirectional Encoder Representations from Transformers. 

BERT models are usually pre-trained on a large corpus of text, then fine-tuned for specific tasks.


In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Set `PATH` to include the directory containing saved_model_cli
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

In [ ]:
import datetime
import shutil

import keras
import keras_hub as hub
import matplotlib.pyplot as plt
import tensorflow as tf
from google.cloud import aiplatform

tf.get_logger().setLevel("ERROR")

To check if you have a GPU attached. Run the following.

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

### Sentiment Analysis

This notebook trains a sentiment analysis model to classify movie reviews as *positive* or *negative*, based on the text of the review.

You'll use the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/).

### Download the IMDB dataset

Let's download and extract the dataset, then explore the directory structure.


In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# Set a path to a folder outside the git repo. This is important so data won't get indexed by git on Jupyter lab
path = "/home/jupyter/"

dataset = keras.utils.get_file(
    "aclImdb_v1.tar.gz", url, untar=True, cache_dir=path, cache_subdir=""
)

dataset_dir = os.path.join(
    os.path.dirname(dataset), "aclImdb_v1_extracted/aclImdb"
)

train_dir = os.path.join(dataset_dir, "train")

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

Next, you will use the `text_dataset_from_directory` utility to create a labeled `tf.data.Dataset`.

The IMDB dataset has already been divided into train and test, but it lacks a validation set. Let's create a validation set using an 80:20 split of the training data by using the `validation_split` argument below.

Note:  When using the `validation_split` and `subset` arguments, make sure to either specify a random seed, or to pass `shuffle=False`, so that the validation and training splits have no overlap.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = keras.utils.text_dataset_from_directory(
    path + "aclImdb_v1_extracted/aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed,
)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = keras.utils.text_dataset_from_directory(
    path + "aclImdb_v1_extracted/aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed,
)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = keras.utils.text_dataset_from_directory(
    path + "aclImdb_v1_extracted/aclImdb/test", batch_size=batch_size
)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Let's take a look at a few reviews.

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f"Review: {text_batch.numpy()[i]}")
        label = label_batch.numpy()[i]
        print(f"Label : {label} ({class_names[label]})")

## Loading models from Keras Hub

For the purpose of this lab, we will be loading a model called Small BERT. Small BERT has the same general architecture as the original BERT but the has fewer and/or smaller Transformer blocks. 

Some other popular BERT models are BERT Base, ALBERT, BERT Experts, Electra. See the continued learning section at the end of this lab for more info. 

Aside from the models available below, there are [multiple versions](https://keras.io/keras_hub/presets/) of the models that are larger and can yeld even better accuracy but they are too big to be fine-tuned on a single GPU. 

You'll see in the code below that switching the tfhub.dev URL is enough to try any of these models, because all the differences between them are encapsulated in the SavedModels from TF Hub.

### Choose a BERT model to fine-tune

In [ ]:
bert_encoder = hub.models.BertBackbone.from_preset(
    "bert_small_en_uncased",
    load_weights=True,
)

print(
    "*******************************\nPre-Trained Model Architecture\n*******************************"
)
bert_encoder.summary()

## Preprocessing model

Text inputs need to be transformed to numeric token ids and arranged in several Tensors before being input to BERT. KerasHub provides a matching preprocessing model for each of the BERT models discussed above, which implements this transformation. It is not necessary to run pure Python code outside your Keras model to preprocess text.

The preprocessing model must be same model as the one that you want to fine-tune in your model later. 

Note: You will load the preprocessing model using a [hub.model.BertPreprocessor](https://keras.io/keras_hub/api/models/bert/bert_text_classifier_preprocessor/) to compose your fine-tuned model.

In [ ]:
bert_preprocessor = hub.models.BertPreprocessor.from_preset(
    "bert_small_en_uncased", name="preprocessor", truncate="waterfall"
)

Let's try the preprocessing model on some text and see the output:

Call the preprocess model function and pass text_test

In [ ]:
text_test = ["this is such an amazing movie!"]
text_preprocessed = bert_preprocessor(text_test)

# This print box will help you inspect the keys in the pre-processed dictionary
print(f"Keys       : {list(text_preprocessed.keys())}")

# 1. token_ids is the ids for the words in the tokenized sentence
print(f'Shape      : {text_preprocessed["token_ids"].shape}')
print(f'Token Ids   : {text_preprocessed["token_ids"][0, :12]}')

# 2. padding_mask is the tokens which we are masking (masked language model)
print(f'Padding Mask : {text_preprocessed["padding_mask"][0, :12]}')

# 3. segment_ids is the sentence id of the input sentence.
print(f'Type Ids   : {text_preprocessed["segment_ids"][0, :12]}')

As you can see, now you have the 3 outputs from the preprocessing that a BERT model would use (`token_ids`, `padding_mask` and `type_ids`).

Some other important points:
- The input is truncated to 512 tokens. 
- The `type_ids` only have one value (0) because this is a single sentence input. For a multiple sentence input, it would have one number for each input.

The text pre-processor is a subclass of `keras.layers.Layer object`. This means that instead of pre-processing separately, we can include it as a layer in the model code.

### Using the BERT model

Before putting BERT into our own model, let's take a look at its outputs.

In [ ]:
bert_encoder = hub.models.BertBackbone.from_preset(
    "bert_small_en_uncased", load_weights=True, name="bert_encoder"
)
bert_encoder.trainable = True

In [ ]:
bert_results = bert_encoder(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

The BERT models return a map with 3 important keys: `pooled_output`, `sequence_output`, `encoder_outputs`:

- `pooled_output` to represent each input sequence as a whole. The shape is `[batch_size, H]`. You can think of this as an embedding for the entire movie review.
- `sequence_output` represents each input token in the context. The shape is `[batch_size, seq_length, H]`. You can think of this as a contextual embedding for every token in the movie review.

For the fine-tuning you are going to use the `pooled_output` array.

### Model Definition

You will create a very simple fine-tuned model, with the preprocessing model, the selected BERT model, one Dense and a Dropout layer.

Note: for more information about the base model's input and output you can use copy the model's url to get to the documentation page.

The order of the layers in the model will be:
1. Input Layer
2. Pre-processing Layer
3. Encoder Layer
4. From the BERT output map, use pooled_output
5. Dropout layer
6. Dense layer with sigmoid activation

In [ ]:
def build_classifier_model(dropout_rate=0.1):
    # 1. Define the input layer for raw text
    text_input = keras.layers.Input(shape=(), dtype="string", name="text")

    # 2. Apply the preprocessor to the input text
    # The 'preprocessor' variable will hold the output dictionary:
    # {'token_ids', 'segment_ids', 'padding_mask'}
    preprocessor = bert_preprocessor(text_input)

    # 3. Load the BERT backbone/encoder
    encoder_outputs = bert_encoder(preprocessor)

    # 4. Use the 'pooled_output' for sentence-level classification
    net = encoder_outputs["pooled_output"]

    # 5.Add any other layers you would like to use
    net = keras.layers.Dropout(dropout_rate, name="dropout")(net)
    net = keras.layers.Dense(1, activation="sigmoid", name="classifier")(net)
    return keras.Model(text_input, net)


dropout_rate = 0.15
classifier_model = build_classifier_model(dropout_rate)
bert_raw_result = classifier_model(keras.ops.convert_to_tensor(text_test))
print(bert_raw_result)

The output is meaningless, of course, because the model has not been trained yet.

Let's take a look at the model's structure.

In [ ]:
keras.utils.plot_model(classifier_model, show_layer_names=True, dpi=100)

## Model training

You now have all the pieces to train a model, including the preprocessing module, BERT encoder, data, and classifier.

### Loss function

Since this is a binary classification problem and the model outputs a probability (a single-unit layer), you'll use `losses.BinaryCrossentropy` loss function.


Define your loss and evaluation metrics here. Since it is a binary classification use BinaryCrossentropy and BinaryAccuracy

In [ ]:
loss = keras.losses.BinaryCrossentropy()
metrics = keras.metrics.BinaryAccuracy()

### Optimizer

For fine-tuning, let's use the same optimizer that BERT was originally trained with: the "Adaptive Moments" (Adam). This optimizer minimizes the prediction loss and does regularization by weight decay (not using moments), which is also known as [AdamW](https://keras.io/api/optimizers/adamw/). AdamW optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments with an added method to decay weights.

In past labs, we have been using the Adam optimizer which is a popular choice. However, for this lab we will be using a variant of Adam called AdamW which is meant to improve generalization using weight decay. The intuition and algoritm behind AdamW can be found in this paper [here](https://arxiv.org/abs/1711.05101).

For the learning rate (`learning_rate`), in line with the BERT paper, the initial learning rate is smaller for fine-tuning (best of 5e-5, 3e-5, 2e-5).

In [ ]:
epochs = 5

optimizer = keras.optimizers.AdamW(learning_rate=3e-5, weight_decay=0.004)

### Loading the BERT model and training

Using the `classifier_model` you created earlier, you can compile the model with the loss, metric and optimizer.

Complile the model using the optimizer, loss and metrics you defined above.

*Note: We are setting an attribute called `jit_compile` here to `False` because of incompatibility with GPU*

In [ ]:
classifier_model.compile(
    optimizer=optimizer, loss=loss, metrics=[metrics], jit_compile=False
)

Note: training time will vary depending on the complexity of the BERT model you have selected.

Let's train the model for a few epochs. Please note that training takes approximately 15-20 mins for 2 epochs when using 1 NVIDIA T4 GPU. 

In [ ]:
print(f"Training model with BERT")
history = classifier_model.fit(
    x=train_ds, validation_data=val_ds, epochs=epochs
)

### Evaluate the model

Let's see how the model performs. Two values will be returned. Loss (a number which represents the error, lower values are better), and accuracy.

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

### Plot the accuracy and loss over time

Based on the `History` object returned by `model.fit()`. You can plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict["binary_accuracy"]
val_acc = history_dict["val_binary_accuracy"]
loss = history_dict["loss"]
val_loss = history_dict["val_loss"]

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, "r", label="Training loss")
# b is for "solid blue line"
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
# plt.xlabel('Epochs')
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, "r", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")

In this plot, the red lines represents the training loss and accuracy, and the blue lines are the validation loss and accuracy.

### Export for inference

Now you just save your fine-tuned model for later use.

To save the model for inference, we will export it to the path specified in SAVEDMODEL_PATH. Keras3 offers methods for saving models, and for this notebook, we are using [model.export()](https://keras.io/api/models/model_saving_apis/export/). This function is chosen because it generates a model in the TensorFlow SavedModel format, which is specifically designed for serving and is compatible with deployment on Google Cloud's Vertex AI Platform.

In [ ]:
dataset_name = "imdb"
saved_model_path = "./{}_bert".format(dataset_name.replace("/", "_"))
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

EXPORT_PATH = os.path.join(saved_model_path, TIMESTAMP)


SAVEDMODEL_PATH = os.path.join(EXPORT_PATH, "savedmodel")

classifier_model.export(SAVEDMODEL_PATH)

Let's reload the model so you can try it side by side with the model that is still in memory. Since we saved the model as a TF SavedModel format, we will be reloading it using `tf.saved_model.load()`.

In [ ]:
reloaded_model = tf.saved_model.load(SAVEDMODEL_PATH)

Here you can test your model on any sentence you want, just add to the examples variable below.

In [ ]:
def print_my_examples(inputs, results):
    result_for_printing = [
        f"input: {inputs[i]:<30} : score: {results[i][0]:.6f}"
        for i in range(len(inputs))
    ]
    print(*result_for_printing, sep="\n")
    print()


examples = [
    "this is such an amazing movie!",  # this is the same sentence tried earlier
    "The movie was great!",
    "The movie was meh.",
    "The movie was okish.",
    "The movie was terrible...",
]

reloaded_results = reloaded_model.serve(keras.ops.convert_to_tensor(examples))
original_results = classifier_model(keras.ops.convert_to_tensor(examples))

print("Results from the saved model:")
print_my_examples(examples, reloaded_results)
print("Results from the model in memory:")
print_my_examples(examples, original_results)

## (Optional) Deploy your model on Vertex AI to get online predictions

We have exported the model to a TensorFlow SavedModel format. Once we have a model in this format, we have lots of ways to "serve" the model, from a web application, from JavaScript, from mobile applications, etc.

Next, print the signature of your saved model using the SavedModel Command Line Interface command saved_model_cli. You can read more about the command line interface and the show and run commands it supports in the [documentation here](https://www.tensorflow.org/guide/saved_model#overview_of_commands).

If you want to use your model on [TF Serving](https://www.tensorflow.org/tfx/guide/serving), remember that it will call your SavedModel through one of its named signatures. In Python, you can test them as follows:

In [ ]:
!saved_model_cli show \
    --tag_set serve \
    --signature_def serving_default \
    --dir {SAVEDMODEL_PATH}

!find {SAVEDMODEL_PATH}
os.environ['SAVEDMODEL_PATH'] = SAVEDMODEL_PATH

In [ ]:
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
PROJECT = !gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"
MODEL_DISPLAYNAME = f"classification-bert-{TIMESTAMP}"

print(f"MODEL_DISPLAYNAME: {MODEL_DISPLAYNAME}")

# from https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-17:latest"
)

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [ ]:
%%bash
# Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "\nHere are your current buckets:"
    gsutil ls
fi

In [ ]:
!gsutil cp -r $SAVEDMODEL_PATH gs://$BUCKET/$MODEL_DISPLAYNAME

We'll upload the model to Vertex AI's model registry. 

In [ ]:
uploaded_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAYNAME,
    artifact_uri=f"gs://{BUCKET}/{MODEL_DISPLAYNAME}",
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
)

We'll create an endpoint and then deploy the model to the endpoint that has been created. 

In [ ]:
MACHINE_TYPE = "n1-standard-4"

endpoint = uploaded_model.deploy(
    machine_type=MACHINE_TYPE,
    accelerator_type=None,
    accelerator_count=None,
)

Once the model has been uploaded to the endpoint, you can query the endpoint to get predictions from the model. The query has to be a list of instances. From the signature of the model we see that the key for the query has to be `text`.

In [ ]:
instances = [
    {"text": ["I loved the movie and highly recomment it"]},
    {"text": ["It was an okay movie in my opinion"]},
    {"text": ["I hated the movie"]},
]

In [ ]:
response = endpoint.predict(instances=instances)

In [ ]:
print(" prediction:", response.predictions)

## Cleanup
When deploying a model to an endpoint for online prediction, the minimum min-replica-count is 1, and it is charged per node hour. So let's delete the endpoint to reduce unnecessary charges. Before we can delete the endpoint, we first undeploy all attached models.

In [ ]:
endpoint.undeploy_all()

...then delete the endpoint.

In [ ]:
endpoint.delete()

#### License

Copyright 2025 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License